In [7]:
import torch
import itertools
import bcolz
import torchvision as tv
import pandas as pd
import numpy as np
import ipywidgets as widgets
import matplotlib.pyplot as plt
from torch.utils.data import dataset, DataLoader, Dataset
from PIL import Image
from functools import partial
from ipywidgets import interact, interactive
from tqdm import tqdm_notebook as tqdm
from tqdm import tnrange
from resnext_101_64x4d import resnext_101_64x4d

In [11]:
def num_features(m):
    c= m if isinstance(m, (list, tuple)) else list(m.children())
    if len(c)==0: return None
    for l in reversed(c):
        if hasattr(l, 'num_features'): return l.num_features
        res = num_features(l)
        if res is not None: return res
        
class AdaptiveConcatPool2d(torch.nn.Module):
    
    def __init__(self, sz=None):
        super().__init__()
        sz = sz or (1,1)
        self.ap = torch.nn.AdaptiveAvgPool2d(sz)
        self.mp = torch.nn.AdaptiveMaxPool2d(sz)
        
    def forward(self, x):
        return torch.cat([self.mp(x), self.ap(x)], 1)

class Lambda(torch.nn.Module):
    
    def __init__(self, f):
        super().__init__(); self.f=f
        
    def forward(self, x):
        return self.f(x)

class Flatten(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, x):
        return x.view(x.size(0), -1)

In [12]:
#resnext_101_64x4d()
model = resnext_101_64x4d()
model.load_state_dict(torch.load('./resnext_101_64x4d.pth'))

front_layers = list(model.children())[:8]
nf = num_features(front_layers) * 2
front_layers += [AdaptiveConcatPool2d(), Flatten()]